<a href="https://colab.research.google.com/github/AstroBesat-SoftW/AstroBesat-SoftW/blob/main/6_versiyon_hata_yuksek___.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [91]:
# DOSYA ADI: teknofest_ozellestirilmis_simulasyon.py
import numpy as np
import pandas as pd
import pickle
import joblib
import xgboost as xgb
import lightgbm as lgb
import os
import tensorflow as tf
import random

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense, Concatenate, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import Callback, EarlyStopping
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, cohen_kappa_score

# Rastgelelikleri sabitle (Tekrarlanabilirlik için)
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

print(">> SİSTEM BAŞLATILIYOR...")
print(">> Hedef: Belirtilen Eğitim (3.940) ve Test (2.460) dağılımlarına göre hazırlık.")

# --- 1. ADIM: ÖZELLEŞTİRİLMİŞ VERİ ÜRETİMİ ---
# Senin verdiğin sayılara birebir uyan veri seti oluşturucu
def kategori_veri_uret(n_patojenik, n_benign, kategori_adi):
    """
    Belirtilen sayıda Patojenik ve Benign veri üretir.
    """
    dna_seqs, prot_seqs, bio_feats, labels = [], [], [], []
    amino_acids = list("ACDEFGHIKLMNPQRSTVWY")

    total = n_patojenik + n_benign

    for i in range(total):
        # Etiket belirle (İlk n_patojenik kadar 1, kalanı 0)
        label = 1 if i < n_patojenik else 0

        # DNA/Protein (Rastgele simülasyon)
        dna = ''.join(np.random.choice(list('ACGT'), size=11))
        prot = ''.join(np.random.choice(amino_acids, size=11))

        # Sayısal Özellikler (Kategoriye göre hafif varyasyonlar eklenebilir ama standart tutuyoruz)
        # Patojenikse (1) risk skoru daha yüksek çıkmaya meyilli olsun diye hile yapmıyoruz,
        # modelin bunu featurelardan öğrenmesini bekliyoruz. Ancak simülasyon olduğu için
        # etiket ile feature arasında korelasyon kuruyoruz.

        if label == 1:
            risk = np.random.beta(5, 2) # Yüksek risk
            cons = np.random.uniform(5, 10)
        else:
            risk = np.random.beta(2, 5) # Düşük risk
            cons = np.random.uniform(0, 5)

        maf = np.random.exponential(0.05)
        hydro = np.random.uniform(-5, 5)
        polar = np.random.uniform(-3, 3)
        weight = np.random.uniform(-50, 50)

        dna_seqs.append(dna)
        prot_seqs.append(prot)
        bio_feats.append([risk, maf, cons, hydro, polar, weight])
        labels.append(label)

    return dna_seqs, prot_seqs, bio_feats, labels

def veri_setini_birlestir(dagilim_sozlugu):
    all_dna, all_prot, all_feats, all_labels = [], [], [], []

    print(f"\n   Veri Dağılımı Oluşturuluyor:")
    for kat_adi, (n_pat, n_ben) in dagilim_sozlugu.items():
        print(f"     - {kat_adi}: {n_pat} Patojenik + {n_ben} Benign = {n_pat+n_ben}")
        d, p, f, l = kategori_veri_uret(n_pat, n_ben, kat_adi)
        all_dna.extend(d)
        all_prot.extend(p)
        all_feats.extend(f)
        all_labels.extend(l)

    # Karıştır (Shuffle)
    combined = list(zip(all_dna, all_prot, all_feats, all_labels))
    np.random.shuffle(combined)
    all_dna, all_prot, all_feats, all_labels = zip(*combined)

    return list(all_dna), list(all_prot), np.array(all_feats).astype('float32'), np.array(all_labels)

# 1.1. EĞİTİM SETİ OLUŞTURMA (3.940 Varyant)
train_dagilim = {
    "Genel Veri Seti": (1500, 1500),
    "Kalıtsal Kanser": (200, 200),
    "Fenilketonüri (PAH)": (200, 200),
    "Kistik Fibrozis (CFTR)": (70, 70)
}
print("\n>> [EĞİTİM SETİ HAZIRLANIYOR]")
dna_train, prot_train, num_train, y_train = veri_setini_birlestir(train_dagilim)
print(f"   >>> TOPLAM EĞİTİM VERİSİ: {len(y_train)} (Beklenen: 3940)")


# 1.2. TEST SETİ OLUŞTURMA (2.460 Varyant)
test_dagilim = {
    "Genel Veri Seti": (1000, 1000),
    "Kalıtsal Kanser": (100, 100),
    "Fenilketonüri (PAH)": (100, 100),
    "Kistik Fibrozis (CFTR)": (30, 30)
}
print("\n>> [TEST SETİ HAZIRLANIYOR - GİZLİ]")
dna_test_ext, prot_test_ext, num_test_ext, y_test_ext = veri_setini_birlestir(test_dagilim)
print(f"   >>> TOPLAM TEST VERİSİ: {len(y_test_ext)} (Beklenen: 2460)")


# --- 2. ADIM: TOKENIZATION VE HAZIRLIK ---
dna_tok = Tokenizer(char_level=True)
dna_tok.fit_on_texts(dna_train) # Sadece Eğitim setini görür!

prot_tok = Tokenizer(char_level=True)
prot_tok.fit_on_texts(prot_train)

# Sequence Dönüşümleri
X_dna_tr_all = pad_sequences(dna_tok.texts_to_sequences(dna_train), maxlen=11, padding='post')
X_prot_tr_all = pad_sequences(prot_tok.texts_to_sequences(prot_train), maxlen=11, padding='post')

X_dna_ext = pad_sequences(dna_tok.texts_to_sequences(dna_test_ext), maxlen=11, padding='post')
X_prot_ext = pad_sequences(prot_tok.texts_to_sequences(prot_test_ext), maxlen=11, padding='post')

# --- 3. ADIM: MODEL İÇİ BÖLÜMLEME (STACKING) ---
# Stacking için eğitim verisini Base (%75) ve Meta (%25) olarak ayırıyoruz.
X_dna_base, X_dna_meta, X_prot_base, X_prot_meta, X_num_base, X_num_meta, y_base, y_meta = train_test_split(
    X_dna_tr_all, X_prot_tr_all, num_train, y_train,
    test_size=0.25, random_state=42
)

# Klasör kontrolü
if not os.path.exists('model_stack'): os.makedirs('model_stack')

# --- 4. ADIM: MODELLERİN EĞİTİMİ ---

# === [1] CNN MODELİ ===
print("\n>> [1/4] CNN Modeli Eğitiliyor...")
in_dna = Input(shape=(11,))
emb_dna = Embedding(len(dna_tok.word_index)+1, 8)(in_dna)
x1 = GlobalMaxPooling1D()(Conv1D(32, 3, activation='relu')(emb_dna))

in_prot = Input(shape=(11,))
emb_prot = Embedding(len(prot_tok.word_index)+1, 8)(in_prot)
x2 = GlobalMaxPooling1D()(Conv1D(32, 3, activation='relu')(emb_prot))

in_num_layer = Input(shape=(6,))
x3 = BatchNormalization()(Dense(32, activation='relu')(in_num_layer))

merged = Concatenate()([x1, x2, x3])
z = Dropout(0.5)(Dense(64, activation='relu')(merged))
out = Dense(1, activation='sigmoid')(z)

model_cnn = Model(inputs=[in_dna, in_prot, in_num_layer], outputs=out)
model_cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Basit bir training (Callback karmaşasını azalttım, odak veri yapısında)
model_cnn.fit(
    [X_dna_base, X_prot_base, X_num_base], y_base,
    validation_data=([X_dna_meta, X_prot_meta, X_num_meta], y_meta),
    epochs=20,
    batch_size=32,
    verbose=0
)
model_cnn.save('model_stack/best_cnn.h5')
print("   >> CNN Tamamlandı.")

# === [2] XGBOOST MODELİ ===
print(">> [2/4] XGBoost Modeli Eğitiliyor...")
model_xgb = xgb.XGBClassifier(n_estimators=100, max_depth=4, eval_metric='logloss', use_label_encoder=False)
model_xgb.fit(X_num_base, y_base)

# === [3] LIGHTGBM MODELİ ===
print(">> [3/4] LightGBM Modeli Eğitiliyor...")
model_lgbm = lgb.LGBMClassifier(n_estimators=100, verbose=-1)
model_lgbm.fit(X_num_base, y_base)

# === [4] STACKING (META-LEARNER) ===
print(">> [4/4] Stacking (Meta Model) Eğitiliyor...")

# Meta seti tahminleri
p_cnn = model_cnn.predict([X_dna_meta, X_prot_meta, X_num_meta], verbose=0).flatten()
p_xgb = model_xgb.predict_proba(X_num_meta)[:, 1]
p_lgbm = model_lgbm.predict_proba(X_num_meta)[:, 1]

X_stack_meta = np.column_stack((p_cnn, p_xgb, p_lgbm))

meta_model = LogisticRegression()
meta_model.fit(X_stack_meta, y_meta)

print(f"   >> Stacking Ağırlıkları -> CNN: {meta_model.coef_[0][0]:.2f}, XGB: {meta_model.coef_[0][1]:.2f}, LGBM: {meta_model.coef_[0][2]:.2f}")

# --- 5. ADIM: FİNAL DEĞERLENDİRME (SENİN TEST SETİNLE) ---
print("\n" + "="*60)
print("🧪 FINAL DEĞERLENDİRME (2.460 Verilik Gizli Test Seti)")
print("="*60)

t_cnn = model_cnn.predict([X_dna_ext, X_prot_ext, num_test_ext], verbose=0).flatten()
t_xgb = model_xgb.predict_proba(num_test_ext)[:, 1]
t_lgbm = model_lgbm.predict_proba(num_test_ext)[:, 1]

X_stack_ext = np.column_stack((t_cnn, t_xgb, t_lgbm))
final_preds = meta_model.predict(X_stack_ext)

f1 = f1_score(y_test_ext, final_preds)
acc = accuracy_score(y_test_ext, final_preds)

print(f"| METRİK               | SONUÇ")
print(f"|----------------------|-----------")
print(f"| F1 Score             | %{f1*100:.2f}")
print(f"| Accuracy             | %{acc*100:.2f}")
print("="*60)

# Modelleri kaydet
joblib.dump(meta_model, 'model_stack/meta_model_final.pkl')
joblib.dump(model_xgb, 'model_stack/xgb_final.pkl')
joblib.dump(model_lgbm, 'model_stack/lgbm_final.pkl')
with open('model_stack/dna_tok_final.pickle', 'wb') as f: pickle.dump(dna_tok, f)
with open('model_stack/prot_tok_final.pickle', 'wb') as f: pickle.dump(prot_tok, f)


# --- 6. ADIM: KULLANICI TAHMİN ARAYÜZÜ ---
# Senin isteğin üzerine: Veri girdiğinde sonucu yazan fonksiyon

def tekli_tahmin_yap(dna, protein, numerical_features):
    """
    Kullanıcıdan gelen tek bir veriyi alır ve eğitilen modellere sorar.
    dna: String (11 karakter)
    protein: String (11 karakter)
    numerical_features: List [Risk, MAF, Cons, Hydro, Polar, Weight]
    """
    print(f"\n❓ YENİ TAHMİN İSTEĞİ: DNA={dna}, PROT={protein}")

    # 1. Hazırlık
    # Tokenizerları yükle (Hafızadakileri kullanıyoruz)
    seq_dna = pad_sequences(dna_tok.texts_to_sequences([dna]), maxlen=11, padding='post')
    seq_prot = pad_sequences(prot_tok.texts_to_sequences([protein]), maxlen=11, padding='post')
    feats = np.array([numerical_features]).astype('float32')

    # 2. Alt Model Tahminleri
    pred_cnn = model_cnn.predict([seq_dna, seq_prot, feats], verbose=0).flatten()[0]
    pred_xgb = model_xgb.predict_proba(feats)[:, 1][0]
    pred_lgbm = model_lgbm.predict_proba(feats)[:, 1][0]

    # 3. Stacking Tahmini
    stack_input = np.array([[pred_cnn, pred_xgb, pred_lgbm]])
    final_prob = meta_model.predict_proba(stack_input)[:, 1][0]
    final_class = meta_model.predict(stack_input)[0]

    # 4. Sonuç Yazdırma
    durum = "PATOJENİK (HASTA)" if final_class == 1 else "BENIGN (İYİ HUYLU)"
    renk = "\033[91m" if final_class == 1 else "\033[92m" # Kırmızı veya Yeşil
    reset = "\033[0m"

    print(f"   -> CNN Güveni : %{pred_cnn*100:.1f}")
    print(f"   -> XGB Güveni : %{pred_xgb*100:.1f}")
    print(f"   -> LGBM Güveni: %{pred_lgbm*100:.1f}")
    print(f"   ----------------------------------")
    print(f"   ► SONUÇ: {renk}{durum}{reset} (Risk Skoru: %{final_prob*100:.2f})")
    return final_class

# ÖRNEK KULLANIM SENARYOSU (Bunu sen verilerini girerek test edebilirsin)
print("\n--- ÖRNEK MANUEL TEST ---")
# Örnek: Patojenik olma ihtimali yüksek veriler verelim
sample_dna = "ACGTACGTACG"
sample_prot = "MMMMMMMMMMM"
# Özellikler: [Risk, MAF, Cons, Hydro, Polar, Weight]
sample_feats = [0.95, 0.01, 9.5, -4.0, 2.5, 10.0]

tekli_tahmin_yap(sample_dna, sample_prot, sample_feats)

# İkinci Örnek: Benign olma ihtimali yüksek veriler
sample_feats_benign = [0.10, 0.40, 1.2, 1.0, 0.5, -10.0]
tekli_tahmin_yap("AAAAAAAAAAA", "LLLLLLLLLLL", sample_feats_benign)

>> AKILLI SİSTEM BAŞLATILIYOR...
>> Ayarlar: Max Epoch=50, Batch=32

   Veri Dağılımı Oluşturuluyor:
     - Genel: 1500 Patojenik + 1500 Benign
     - Kanser: 200 Patojenik + 200 Benign
     - PAH: 200 Patojenik + 200 Benign
     - CFTR: 70 Patojenik + 70 Benign

   Veri Dağılımı Oluşturuluyor:
     - Genel: 1000 Patojenik + 1000 Benign
     - Kanser: 100 Patojenik + 100 Benign
     - PAH: 100 Patojenik + 100 Benign
     - CFTR: 30 Patojenik + 30 Benign

>> [1/4] CNN Modeli: İdeal Epoch Hesaplanıyor...


   [Epoch 1/50] -> Val F1: %82.29 ⭐ (YENİ REKOR! Kaydediliyor...)


   [Epoch 2/50] -> Val F1: %90.66 ⭐ (YENİ REKOR! Kaydediliyor...)


   [Epoch 3/50] -> Val F1: %97.44 ⭐ (YENİ REKOR! Kaydediliyor...)


   [Epoch 4/50] -> Val F1: %99.20 ⭐ (YENİ REKOR! Kaydediliyor...)


...

/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [01:38:01] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


>> [4/4] Stacking (Meta Model) Eğitiliyor...


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(



🧪 FINAL DEĞERLENDİRME (Gizli Test Seti)
| METRİK               | SONUÇ
|----------------------|-----------
| F1 Score             | %99.96
| Accuracy             | %99.96
| Kappa Score          | 0.9992

--- TEST: Örnek Veri ---

🔎 Girdi: ACGTACGTACG | MMMMMMMMMMM
   Sonuç: PATOJENİK (Güven: %99.4)


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [100]:
# DOSYA ADI: canli_test_simulasyonu.py
import numpy as np
import pandas as pd
import pickle
import joblib
import time
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Renk Kodları (Görsellik için)
YESIL = "\033[92m"
KIRMIZI = "\033[91m"
SARI = "\033[93m"
RESET = "\033[0m"

print(f"{SARI}>> SİSTEM BAŞLATILIYOR... MODELLER YÜKLENİYOR...{RESET}")

# --- 1. MODELLERİ YÜKLE ---
try:
    with open('model_stack/dna_tok_final.pickle', 'rb') as f: dna_tok = pickle.load(f)
    with open('model_stack/prot_tok_final.pickle', 'rb') as f: prot_tok = pickle.load(f)
    model_cnn = load_model('model_stack/best_cnn.h5')
    model_xgb = joblib.load('model_stack/xgb_final.pkl')
    model_lgbm = joblib.load('model_stack/lgbm_final.pkl')
    meta_model = joblib.load('model_stack/meta_model_final.pkl')
    print("✅ Modeller hafızaya alındı.\n")
except:
    print("❌ HATA: Modeller bulunamadı. Lütfen önce eğitim kodunu çalıştırın.")
    exit()

# --- 2. RASTGELE VERİ ÜRETİCİ (GERÇEK ETİKETLİ) ---
def rastgele_veri_uret():
    """
    Tek bir satır rastgele veri üretir ve matematiksel kurala göre
    bunun GERÇEKTE ne olması gerektiğini belirler.
    """
    amino_acids = list("ACDEFGHIKLMNPQRSTVWY")

    # 1. Rastgele Diziler
    dna = ''.join(np.random.choice(list('ACGT'), size=11))
    prot = ''.join(np.random.choice(amino_acids, size=11))

    # 2. Rastgele Sayısal Özellikler
    # Simülasyon gereği: Risk ve Cons yüksekse Patojenik olma ihtimali artar.
    # Modelin bunu bilip bilmediğini test edeceğiz.

    risk = np.random.beta(2, 2) # 0 ile 1 arası rastgele
    maf = np.random.exponential(0.05)
    cons = np.random.uniform(0, 10)
    hydro = np.random.uniform(-5, 5)
    polar = np.random.uniform(-3, 3)
    weight = np.random.uniform(-50, 50)

    # 3. GERÇEK ETİKETİ HESAPLA (Simülasyon Kuralı)
    # Bu formül doğanın kanunu gibi davranır. Model bunu bilmiyor, tahmin etmeye çalışıyor.
    score = (risk * 0.4) + (cons/10 * 0.2) + ((0.5 - maf)*2 * 0.2) + (abs(hydro)/5 * 0.2)
    score += np.random.normal(0, 0.05) # Biraz gürültü ekle

    gercek_etiket = 1 if score > 0.65 else 0

    features = [risk, maf, cons, hydro, polar, weight]
    return dna, prot, features, gercek_etiket

# --- 3. TAHMİN FONKSİYONU ---
def model_tahmin_et(dna, prot, features):
    # Hazırlık
    seq_dna = pad_sequences(dna_tok.texts_to_sequences([dna]), maxlen=11, padding='post')
    seq_prot = pad_sequences(prot_tok.texts_to_sequences([prot]), maxlen=11, padding='post')
    x_num = np.array([features]).astype('float32')

    # Tahminler
    p_cnn = model_cnn.predict([seq_dna, seq_prot, x_num], verbose=0).flatten()[0]
    p_xgb = model_xgb.predict_proba(x_num)[:, 1][0]
    p_lgbm = model_lgbm.predict_proba(x_num)[:, 1][0]

    # Stacking
    stack_input = np.array([[p_cnn, p_xgb, p_lgbm]])
    final_pred_class = meta_model.predict(stack_input)[0]
    final_prob = meta_model.predict_proba(stack_input)[:, 1][0]

    return final_pred_class, final_prob

# --- 4. CANLI TEST DÖNGÜSÜ ---
print(f"{'DNA':<12} {'PROTEİN':<12} {'RİSK':<6} | {'GERÇEK':<10} | {'TAHMİN':<10} | {'SONUÇ'}")
print("-" * 75)

dogru_sayisi = 0
toplam_test = 20  # Kaç tane test yapsın?

for i in range(toplam_test):
    # 1. Veri Üret
    dna, prot, feats, gercek = rastgele_veri_uret()
    risk_val = feats[0]

    # 2. Modele Sor
    tahmin, olasilik = model_tahmin_et(dna, prot, feats)

    # 3. Karşılaştır
    durum_gercek = "HASTA" if gercek == 1 else "SAĞLIKLI"
    durum_tahmin = "HASTA" if tahmin == 1 else "SAĞLIKLI"

    # Sonuç Stringi Hazırla
    if gercek == tahmin:
        sonuc_ikon = f"{YESIL}✅ DOĞRU{RESET}"
        dogru_sayisi += 1
    else:
        sonuc_ikon = f"{KIRMIZI}❌ YANLIŞ{RESET}"

    # Renkli çıktı için
    c_gercek = KIRMIZI if gercek == 1 else YESIL
    c_tahmin = KIRMIZI if tahmin == 1 else YESIL

    print(f"{dna} {prot} {risk_val:.2f}   | {c_gercek}{durum_gercek:<10}{RESET} | {c_tahmin}{durum_tahmin:<10}{RESET} | {sonuc_ikon}")

    # Küçük bir bekleme efekti (İstersen kaldırabilirsin)
    time.sleep(0.1)

# --- 5. ÖZET RAPOR ---
print("-" * 75)
basari_orani = (dogru_sayisi / toplam_test) * 100
print(f"\n📊 TOPLAM SONUÇ:")
print(f"   Test Edilen Veri Sayısı : {toplam_test}")
print(f"   Doğru Bilinen           : {dogru_sayisi}")
print(f"   Yanlış Bilinen          : {toplam_test - dogru_sayisi}")
print(f"   MODEL BAŞARISI          : {SARI}%{basari_orani:.2f}{RESET}")

if basari_orani > 90:
    print(f"   YORUM: {YESIL}Mükemmel! Modelin teşhis yeteneği çok yüksek.{RESET}")
elif basari_orani > 75:
    print(f"   YORUM: {SARI}Gayet iyi. Yarışma için yeterli seviyede.{RESET}")
else:
    print(f"   YORUM: {KIRMIZI}Model biraz kararsız. Eğitimi tekrar etmen gerekebilir.{RESET}")

>> SİSTEM BAŞLATILIYOR... MODELLER YÜKLENİYOR...
✅ Modeller hafızaya alındı.

DNA          PROTEİN      RİSK   | GERÇEK     | TAHMİN     | SONUÇ
---------------------------------------------------------------------------


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


GACGGAATTAT RFESCYNTHVW 0.66   | HASTA      | SAĞLIKLI   | ❌ YANLIŞ


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


AACCCACGTAT GAGWNRMRDFA 0.50   | HASTA      | HASTA      | ✅ DOĞRU


...

TCTAGCCACGA MMFTEQNCPKE 0.21   | SAĞLIKLI   | SAĞLIKLI   | ✅ DOĞRU
GAAATAGTCTT GQFPISGVYER 0.89   | SAĞLIKLI   | HASTA      | ❌ YANLIŞ


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


TGATTTTGCCG TLFGTCPEVYC 0.30   | SAĞLIKLI   | HASTA      | ❌ YANLIŞ


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


AGCCTATGCAA TFEAMSERMEE 0.45   | SAĞLIKLI   | HASTA      | ❌ YANLIŞ
CACGTATAAAG SPMWTLVSIYD 0.05   | SAĞLIKLI   | HASTA      | ❌ YANLIŞ
---------------------------------------------------------------------------

📊 TOPLAM SONUÇ:
   Test Edilen Veri Sayısı : 20
   Doğru Bilinen           : 9
   Yanlış Bilinen          : 11
   MODEL BAŞARISI          : %45.00
   YORUM: Model biraz kararsız. Eğitimi tekrar etmen gerekebilir.
